In [ ]:
## One-Dim Incremental learning

In [3]:
from typing import ForwardRef
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F
from torch.autograd import Variable

class OneDimNet(nn.Module):
    def __init__(self):
        super(OneDimNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=10, kernel_size=3, stride=2) # length = 479
        self.max_pool1=nn.MaxPool1d(kernel_size=3, stride=2) # length = 239
        self.conv2=nn.Conv1d(10, 20, 3, 2) # length = 119
        self.max_pool2 = nn.MaxPool1d(3, 2) # length = 59
        self.conv3=nn.Conv1d(20, 40, 3, 2) # length = 29

        self.liner1 = nn.Linear(40 * 14, 120)
        self.liner2 = nn.Linear(120, 84)
        self.liner3 = nn.Linear(84, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        

        x = x.view(-1, 40 * 14)
        x = F.relu(self.liner1(x))
        x = F.relu(self.liner2(x))
        x = self.liner3(x)

        return x

In [4]:
conv1=nn.Conv1d(20, 40, 3, 2) # length = 29
input = torch.randn(1, 29, 20) # batch, length, channels

In [5]:
input = input.permute(0,2,1)
input = Variable(input)

In [6]:
out = conv1(input)

In [7]:
out.size()

torch.Size([1, 40, 14])

In [8]:
sin = np.array(np.load('sin.npy'))
tri = np.array(np.load('tri.npy'))
squ = np.array(np.load('squ.npy'))
x = np.array([sin,tri,squ])
x.shape
y = np.array([[0],[1],[2]])
y
y.shape


FileNotFoundError: [Errno 2] No such file or directory: 'sin.npy'

In [ ]:
## 这里写了一个简单的神经网络，方便理解

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
# 效果不太好
sin = np.array(np.load('data/sin.npy'))
tri = np.array(np.load('data/tri.npy'))
squ = np.array(np.load('data/squ.npy'))

sin_30 = np.array(np.load('data/sin_30.npy'))
tri_30 = np.array(np.load('data/tri_30.npy'))
squ_30 = np.array(np.load('data/squ_30.npy'))
# 构建输入集
# x = np.mat('0 0;'
#            '0 1;'
#            '1 0;'
#            '1 1')

x = np.array([sin,tri,squ,sin_30,tri_30,squ_30])
x = torch.tensor(x).float()
y = np.array([[0],[1],[2],[0],[1],[2]])
y = torch.tensor(y).float()



# 搭建网络
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cl1 = nn.Linear(256, 128)
        self.cl2 = nn.Linear(128,10)
        self.fc1 = nn.Linear(10,1)
    
    def forward(self, x):
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.fc1(x))
        return x

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

myNet = MyModel()
myNet.fc1.register_forward_hook(get_activation('fc1'))

# 设置优化器
optimzer = torch.optim.SGD(myNet.parameters(), lr=0.05)
loss_func = nn.MSELoss()


for epoch in range(5000):
    out = myNet(x)
    loss = loss_func(out, y)  # 计算误差
    optimzer.zero_grad()  # 清除梯度
    loss.backward()
    optimzer.step()

# print(activation['fc1'])

print(myNet(x).data)



tensor([[0.],
        [1.],
        [2.],
        [0.],
        [1.],
        [2.]])


In [9]:
x = np.array(np.load('data/sin_31.npy'))
x = torch.tensor(x).float()
print(myNet(x).data)

tensor([0.])


In [10]:
a = []
a.append(2)
a

[2]

In [11]:
# from typing import ForwardRef
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F
from torch.autograd import Variable

class OneDimNet(nn.Module):
    def __init__(self):
        super(OneDimNet, self).__init__()
        ## input = 1 x 227 x 1 (Row, Column, channel)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0) # weight of size [96, 1, 11] (out chan, in chan, ker_size)
        self.max_pool1=nn.MaxPool1d(kernel_size=3, stride=2) # length = 
        self.conv2=nn.Conv1d(96, 256, 5, 1, 2) # length = 119
        self.max_pool2 = nn.MaxPool1d(3, 2) # length = 59
        self.conv3=nn.Conv1d(256, 384, 3, 1, 1) # length = 29
        self.conv4=nn.Conv1d(384, 384, 3, 1, 1)
        self.conv5=nn.Conv1d(384, 256, 3, 1, 1)


        self.fc1 = nn.Linear(256, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.liner1(x))
        x = F.relu(self.liner2(x))
        x = self.liner3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features

    # optimzer = torch.optim.SGD(OneDimNet.parameters(), lr=0.05)
    # loss_func = nn.MSELoss()

    # for epoch in range(5000):
    #     out = OneDimNet(x)
    #     loss = loss_func(out, y)  # 计算误差
    #     optimzer.zero_grad()  # 清除梯度
    #     loss.backward()
    #     optimzer.step()

In [12]:
net=OneDimNet()
print(net)

input = Variable(torch.randn(3, 1, 227)) # batch, channel, Height, Width

optimzer = torch.optim.SGD(OneDimNet.parameters(), lr=0.05)
loss_func = nn.MSELoss()

for epoch in range(5000):
    out = OneDimNet(x)
    loss = loss_func(out, y)  # 计算误差
    optimzer.zero_grad()  # 清除梯度
    loss.backward()
    optimzer.step()

OneDimNet(
  (conv1): Conv1d(1, 96, kernel_size=(11,), stride=(4,))
  (max_pool1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(96, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (max_pool2): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 384, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv5): Conv1d(384, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=256, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=3, bias=True)
)


TypeError: parameters() missing 1 required positional argument: 'self'

In [13]:
conv1 = nn.Conv1d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0) 
input = Variable(torch.randn(1, 1, 227)) # batch, channel, Height, Width
out = conv1(input)
print(out)

tensor([[[ 0.4788, -0.4595,  0.4206,  ...,  0.6415,  0.8621, -0.0820],
         [ 0.9075,  0.7026, -0.4566,  ...,  0.8942, -0.3017, -0.0664],
         [ 0.3873,  0.1756,  0.7099,  ...,  1.2519,  0.8428,  0.8790],
         ...,
         [ 0.4584,  0.7585,  0.8612,  ..., -0.2446, -0.0886,  0.2389],
         [ 0.0897, -0.3149, -0.0532,  ..., -0.5106, -1.0129, -0.7503],
         [-0.5296,  0.0194, -0.7537,  ...,  0.4335, -0.3794, -1.6667]]],
       grad_fn=<SqueezeBackward1>)
